In [78]:
%pylab inline
from IPython.display import Image, JSON, Latex, Math, Pretty, SVG

Populating the interactive namespace from numpy and matplotlib


# Object Oriented Programming Tutorial



Justin Lanfranchi



2016.05.17


# Outline / Concepts

* "Good programming"
* Procedural programming
* A concrete example: Data acquisition
* Object oriented programming
* Class
* Object
* Polymorphism
* Inheritance

# "Good" programming doesn't require one paradigm or another

except in narrow circumstances.

But certain principles of programming should be followed, no matter what programming paradigm you're using. And object oriented programming makes following some of these principles more natural (well, in *principle*...).

## The principles of good programming

* Don't repeat yourself

* Don't repeat yourself

## Secondary principles of good programming

* Find someone else's implementation of the thing *(i.e.... don't repeat (someone else's) code)*
  * But prove that it works correctly, especially math/science codes

* Consider how your software will be used, not just by you
  * Think in terms of *interfaces*: the implementation can change without breaking other code that makes use of it.
  * Suppose you find a faster way to do procedure A or implement object X. This shouldn't break all code that calls procedure A or makes use of object X.

* Encapsulate complexity and hide the details that a user of that part of the software shouldn't have to worry about.

* Write code that you'll understand after not using for 2 years.

* *"Always code as if the guy who ends up maintaining your code will be a violent psychopath who knows where you live. Code for readability."* -John F. Woods (or somebody)

* Abstract what you're trying to accomplish enough that you can use your code to do something similar.

* Don't abstract so much that you can do *anything* with your code. Then you've done nothing at all.

## Vague, unproven programming wisdom

* The better you understand datastructures, the better you'll code


* Try to draw the problem you're trying to solve & its solution as block diagrams


* You never get it right the first time anyway, so don't try to be perfect the first time.


* Stackoverflow collects too much wisdom to ignore; use it shamelessly (but understand the tradeoffs discussed there)


* Others in the group are here to help, ask us and others questions frequently


* Use the Collie Method to debug your code

# Procedural programming

Put statements together into a function to execute them together.

Possibly have inputs, and possibly have outputs.

In [165]:
# Define a function "func"
def func(x):
    print 'x =', x
    y = x**2
    print 'y = x**2 =', y
    return y

# Run the function on data
mydata = 2.3
rslt = func(2.3)
print 'rslt =', rslt

x = 2.3
y = x**2 = 5.29
rslt = 5.29


* Quite natural for certain operations

  * Particularly if the "state" is a simple thing (a number, an array, etc.)


* Typically the first paradigm of programming you learn


* Python is perfectly happy if you program procedurally.


Note: Python itself implements an object model for the things in the language, so you'll encounter object oriented concepts whether you realize it or not while you use Python.

In [14]:
def square(x):
    new_list = []
    for val in x:
        new_list.append(val**2)
    return new_list

In [15]:
vals = [1, 10.2, 18]
sq_vals = square(vals)
sq_vals

[1, 104.03999999999999, 324]

In [17]:
def mean(x):
    return sum(x)/len(x)

def root(x):
    return sqrt(x)

In [18]:
rms = root(mean(square(vals)))
rms

11.958818224780128

# A Concrete Example to Follow

![worlds worst diagram](img/lab_setup.png)

![Another worlds worst illustration](img/lab_setup_specan.png)

Communications link can be

* Serial (RS-232)
* USB
* Ethernet
* ...

### Abstraction
Whatever kind of communications interface and whatever kind of equipment, we are still just sending messages to the equipment and receiving messages (and data) back from the equipment. This is the same for the scope and for the specan, same for RS-232, USB, and Ethernet. So this seems like a good candidate for a task that can be abstracted.

### Consider how your software will be used
At first glance, as a user, I want to be able to do several very basic things:
1. Query the equipment's state
2. Set the equipment's state
3. Request that it take data
4. Save the data to disk

1-3 above all make use of the communications interface, so we'll make sure our architecture handles those things for all interfaces. And the operations are just the same for both pieces of equipment, at least at this level we're abstracting it to.

Thinking a bit more, we realize we might want to manipulate the data, display the data, and *then* store the data to disk. Or some ordering thereof.

Do we know exactly how the data needs to be manipulated yet? Do we know exactly how we want to plot the data? Well... (long discussion ensues)...

**Stop!**

Let's get the well-defined parts of the process built and working, and then we can figure out the rest. Those will probably be separate pieces since they are doing different things (analyzing the data vs. getting the data), so let's leave that aside for now.

... so we come back here:

### Consider how your software will be used
At first glance, as a user, I want to be able to do several very basic things:
1. Query the equipment's state
2. Set the equipment's state
3. Request that it take data
4. Save the data to disk

1-3 above all make use of the communications interface, so we'll make sure our architecture handles those things for all interfaces. And the operations are just the same for both pieces of equipment, at least at this level we're abstracting it to.

### Encapsulate and hide complexity
The oscilloscope has certain parameters that the user might change frequently:
* Sample rate
* Duration of data-taking
* Lowpass filter cutoff frequency
* ...

You want to **expose these** to the user.

The scope also has parameters that will be changed only occasionally by the user:
* Baud rate for communications
* IP address / serial port
* ...

You can **encapsulate these** in the form of a configuration file, and the user just has to specify the config file rather than all of these details every time.

And finally the scope has parameters or details that the user simply doesn't care about:
* Scope's command set and which command gets sent when
* Data format and encoding of messages sent/received
* Besides initially setting up the communications protocol, the user doesn't care which one the equipment uses
* ...

These should be **hidden** from the user.

Likewise, your software has many details that the user shouldn't have to worry about:
* Did you use a list or a numpy array to hold the data?
* Is the equipment ID the 3rd or 4th item in the list that contains equipment metadata?
* Which communications protocol is being called by the software?

These complexities, too, should be **hidden** from the user.

# Object Oriented Programming

Objects are things that have both *methods* (i.e. actions, or things they can *do*) as well as *data* (or attributes, state, ... however you want to see it; it remembers stuff).

Methods are just like functions in procedural programming, but they can access the state of the object they live in.

The fact that objects hold "data" (i.e., the "state" of an object) makes them more like real things we interact with.

# Class

A ***class*** is the **definition of an object**.

A class is not the object, it is the description of the object.

## Start with the communications interface

We know that different pieces of equipment use different comms interfaces. This can be one of several different kinds (e.g., RS-232, USB, and/or Ethernet).

Below we'll define a ***base class***.

In [188]:
class CommsInterface(object):
    def __init__(self):
        self.send_message_queue = []
        self.receive_message_queue = []
        self.__is_initialized = False
        self._did_something = False
        
        self.do_something()
    
    @property
    def did_something(self):
        return self._did_something
    
    @did_something.setter
    def did_something(self, value):
        if not isinstance(value, bool):
            raise TypeError('`value` must be bool, fool!')
        self._did_something = value
    
    def initialize_interface(self):
        if not self.is_initialized:
            self._initialize_interface()
    
    def teardown_interface(self):
        if self.is_initialized:
            self._teardown_interface()
    
    def write(self, message, timeout=20):
        self.initialize_interface()
        self._write(message=message, timeout=timeout)
        
    def read(self, length=0, timeout=1):
        self.initialize_interface()
        self._read(length=length, timeout=timeout)
    
    def _initialize_interface(self):
        raise NotImplementedError('You have to override this method in your subclass!')
        
    def _write(self, message, timeout):
        raise NotImplementedError('You have to override this method in your subclass!')
        
    def _read(self, length, timeout):
        raise NotImplementedError('You have to override this method in your subclass!')
    
    def do_something(self):
        self._did_something = True

## Object

An ***object*** is the instance of a class.

It's a real thing that can *actually* be interacted with, and that *actually* holds data.

$\int x \; dx$

In [189]:
# "Instantiate" an object
interface = CommsInterface()

In [190]:
interface.did_something

True

In [191]:
interface.did_something = 'stupid things'

TypeError: `value` must be bool, fool!

In [193]:
interface.did_something = False
print interface.did_something

False


In [181]:
interface._did_something = 'stupid things'
interface._did_something

'stupid things'

In [182]:
interface._CommsInterface__is_initialized = 'more stupid things'
interface._CommsInterface__is_initialized

'more stupid things'

In [139]:
# What "type" is the object?
type(interface)

__main__.CommsInterface

In [140]:
# Check explicitly in your code...
if isinstance(interface, CommsInterface):
    print 'the object named "interface" *is* a CommsInterface!'

the object named "interface" *is* a CommsInterface!


In [141]:
x = 123
isinstance(x, CommsInterface)

False

In [142]:
# Access attributes of the object
interface._

False

In [143]:
interface.did_something

True

In [144]:
# Set a new attribute of the object
interface.newattr = 'this is an attribute not defined in the class'

print interface.newattr

this is an attribute not defined in the class


In [145]:
# Overwrite an attribute
interface.is_initialized = 'this should not be allowed!'

print interface.is_initialized

this should not be allowed!


In [146]:
# Try to do something with it
interface.read()

NotImplementedError: You have to override this method in your subclass!

In [147]:
# Try to do something with it again, but catch the exception
try:
    interface.read()
except NotImplementedError:
    print 'not implemented, but allowing the program to go on anyway...'
    
x = 'blah'
print x

not implemented, but allowing the program to go on anyway...
blah


# Inheritance

If a class B ***inherits*** from class A, then it gets all of the properties and methods of A.

Class A is said to be the ***base class*** or the ***superclass*** of class B.

We will now define the class RS232 which *inherits* from the CommsInterface class.

In [154]:
class RS232(CommsInterface):
    def __init__(self, com_port):
        super(RS232, self).__init__()
        self.com_port = com_port
        
    def _initialize_interface(self):
        self.serial_port = serial.open(self.com_port)
        self.is_initialized = True
        
    def _write(self, message, timeout):
        self.serial_port.write(message, timeout=timeout)
        
    def _read(self, timeout):
        self.serial_port.read(timeout=timeout)

In [155]:
interface = RS232('com1')
isinstance(interface, RS232)

True

In [156]:
isinstance(interface, CommsInterface)

True

### Polymorphism:
Derived classes being able to define their own methods, as in the `RS232` class above.

### Python has special methods to work with objects

An important one to know is `__dict__` (among many others you'll learn later).

`__dict__` tells you the attributes of an object.

In [157]:
interface.__dict__

{'com_port': 'com1',
 'did_something': True,
 'is_initialized': False,
 'receive_message_queue': [],
 'send_message_queue': []}

With ipython, you can also type

  `interface.<tab>`

and it will display the attributes and methods of an object.

### Now define a new derived class, Ethernet

This one will *not* call `super(...).__init__()`

In [164]:
class Ethernet(CommsInterface):
    def __init__(self, ip_address, port):
        self.ip_address = ip_address
        self.port = port
        
    def _initialize_interface(self):
        # ...
        self.is_initialized = True

In [159]:
interface = Ethernet('192.168.0.1', 8888)

In [162]:
print isinstance(interface, Ethernet)
print isinstance(interface, CommsInterface)

True
True


In [163]:
interface.__dict__

{'ip_address': '192.168.0.1', 'port': 8888}

# Notes about OO

### Find out about an object directly through its attributes.

If you have 6 of the same kind of thing, say 6 Polygon objects.

Rather than keeping track of the 6 color and 6 shape attributes in two lists that you try to keep up to date...


just need to keep track of the 6 objects, and you can always ask for the objects to tell you about their colors and shapes at any point.


This keeps the "metadata" about the objects in sync with the actual state of those objects.

# Particulars of Python

## Most things are passed by reference ("pointers")

This means that all but the simplest datastructures don't get copied into functions/methods, but just a pointer to the data is passed around. Pointer = its address.

Calling a function or method with an object in its arguments:
* *"Please operate on the piece of data that lives at 201 Main St."*


Then if the function does something to modify the data, it's modifying the data that actually lives at 201 Main St.

In [207]:
# Create a simple function (works same for methods of a class)
def func(var):
    print 'the arg I got lives at\n\t', id(var)
    original = var
    var[0] = 3
    #our_list[1] = 'something else'
    print 'var is', var

In [208]:
# Create a list named "x"
x = (['a','b'], 3, -2, 4)
print id(x)
print '  x =', x, '(before)'

140457198674088
  x = (['a', 'b'], 3, -2, 4) (before)


In [213]:
def func(*args):
    print args

# build up an argument list
arg_list = []
arg_list.append(0)
arg_list.append(10)
print arg_list


[0, 10]


In [217]:
func(*['a', 'b', 'c'])

('a', 'b', 'c')


In [209]:
func(x)

the arg I got lives at
	140457198674088


TypeError: 'tuple' object does not support item assignment

In [201]:
print 'x is', x, '(after)'

x is [3, 3, -2, 4] (after)


If you don't want this behavior, you can create a physical copy the object, and then manipulate the copy.

In [219]:
from copy import copy, deepcopy

def func2(var):
    print 'the arg I got lives at\n\t', id(var)
    original = var
    var = deepcopy(original)
    print 'the copy of the arg I got lives at\n\t', id(var)
    var[0] = 3
    print 'var =', var

In [220]:
x = [1, 3, -2, 4]
print 'x lives at\n\t', id(x)
print 'x is', x, '(before)'

x lives at
	140457198041928
x is [1, 3, -2, 4] (before)


In [221]:
func2(x)

the arg I got lives at
	140457198041928
the copy of the arg I got lives at
	140457198006072
var = [3, 3, -2, 4]


In [222]:
print 'x is', x, '(after)'

x is [1, 3, -2, 4] (after)


This is true of everything but "immutable objects." Immutable objects are simple numbers float, int, bool, and tuples. These are passed "by value," i.e., their actual value is passed to the function/method.

In [98]:
def func3(var):
    var = 2
    print 'var =', var

In [99]:
x = (1, 5, 6)
print 'x =', x, '(before)'

x = (1, 5, 6) (before)


In [100]:
func3(x)

var = 2


In [101]:
print 'x =', x, '(after)'

x = (1, 5, 6) (after)


In [12]:
    
class Instrument:
    """Control lab equipment.
    
    Parameters
    ----------
    interface : CommsInterface
        Interface to use to talk to the instrument. Must
        be an instantiated CommsInterface object.
    
    """
    def __init__(self, interface):
        # These are "state" or "data" that will be attached to the object
        self.interface_type = interface_type
        self.interface_parameters = interface_parameters
        
        # Call methods that are defined within the class...
        self.initialize_interface()
        self.initialize_instrument()
        
    def initialize_interface(self):
        if self.interface_type == 'rs-232':
            self.interface = RS232()
    def initialize_instrument(self):
        gpib.open(self.address)
        gpib.write('open')
        
    def take_data(self, points):
        self.

IndentationError: expected an indented block (<ipython-input-12-2a4796622f2f>, line 29)